<a href="https://colab.research.google.com/github/TalhaUsuf/Angular-Penalty-Softmax-Losses-Pytorch/blob/master/TripletMarginLossMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

     |████████████████████████████████| 106 kB 14.8 MB/s 
     |████████████████████████████████| 85.5 MB 76.4 MB/s 


In [2]:
!nvidia-smi

Fri Jan 14 15:43:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install gpustat

     |████████████████████████████████| 78 kB 4.3 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=dddbd7f9c75e0196e50298ed3035242d44bd4158890886f6ea6766d0953fba74
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a93930798b8
Successfully built gpustat


In [4]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
!pip install rich
!pip install pytorch_lightning
from rich.console import Console


     |████████████████████████████████| 215 kB 14.6 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 
     |████████████████████████████████| 526 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 71.0 MB/s 
     |████████████████████████████████| 829 kB 84.3 MB/s 
     |████████████████████████████████| 332 kB 87.5 MB/s 
     |████████████████████████████████| 133 kB 101.4 MB/s 
     |████████████████████████████████| 1.1 MB 74.0 MB/s 
     |████████████████████████████████| 271 kB 97.1 MB/s 
     |████████████████████████████████| 160 kB 86.9 MB/s 
     |████████████████████████████████| 192 kB 92.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d8b37fa4aed82d5a7fce7c8833f184bec23772a153c19e04673620eea1d050fe
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
  

In [5]:
!pip install wandb

In [ ]:
# import pytorch_lightning as pl


# class LitModel(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         self.l1 = nn.Linear(28 * 28, 10)

#     def forward(self, x):
#         return torch.relu(self.l1(x.view(x.size(0), -1)))

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         return loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pytorch_lightning import LightningModule, Trainer

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import wandb







class recognition_model(LightningModule):
    def __init__(self, trainset, testset, conv1_N = 32, conv1_filter = 3 , conv1_stride = 1, 
                 conv2_N = 64, conv2_filter = 3 , conv2_stride = 1 , dropout = 0.25, linear = [128, 512, 128], lr = 0.0001):
        super(recognition_model, self).__init__()
        self.train_set = trainset
        self.test_set = testset
        self.lr = lr
        self.save_hyperparameters()
        self.conv1 = nn.Conv2d(1, conv1_N, conv1_filter, conv1_stride) # [N, 30, 30, 32]
        self.conv2 = nn.Conv2d(conv1_N, conv2_N, conv2_filter, conv2_stride) # [N, 28, 28, 64]
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(33856, linear[0])
        self.fc2 = nn.Linear(linear[0], linear[1])
        self.fc3 = nn.Linear(linear[1], linear[2])                
        

        # metrics
        self.distance = distances.CosineSimilarity()
        self.reducer = reducers.ThresholdReducer(low=0)
        self.loss_func = losses.TripletMarginLoss(margin=0.2, distance=self.distance, reducer=self.reducer)
        self.mining_func = miners.TripletMarginMiner(
            margin=0.2, distance=self.distance, type_of_triplets="semihard"
        )
        self.accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)

    def forward(self, x):
        x = self.conv1(x)# [N, 30, 30, 32]
        x = F.relu(x)# [N, 30, 30, 32]
        x = self.conv2(x)# [N, 28, 28, 64]
        x = F.relu(x)# [N, 28, 28, 64]
        x = F.avg_pool2d(x, 2, 1)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        # Console().rule(title=f"after flatten ===> {x.shape}", style="red on black")
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        x = F.gelu(x)
        x = self.fc3(x)
        return x

    def get_all_embeddings(self, dataset, model):
        tester = testers.BaseTester()
        return tester.get_all_embeddings(dataset, model)

    def training_step(self, batch, batch_idx):

        data, labels = batch
        embeddings = self.forward(data)
        # Console().rule(title=f"EMBEDDINGS shape ===> {embeddings.shape}", style="bold magenta")
        indices_tuple = self.mining_func(embeddings, labels)
        # Console().rule(title=f"INDICES shape ===> {indices_tuple}", style="bold magenta")
        loss = self.loss_func(embeddings, labels, indices_tuple)
        # Console().print(f"epoch --> {self.current_epoch}\t\t Loss --> {loss.tolist()}", style="bold cyan")
        self.log(
            "train_loss" , loss.tolist()
            )
        
        return {"loss" : loss}
    
    def validation_step(self, batch, batch_idx):

        train_embeddings, train_labels = self.get_all_embeddings(self.train_set, self)
        test_embeddings, test_labels = self.get_all_embeddings(self.test_set, self)
        train_labels = train_labels.squeeze(1)
        test_labels = test_labels.squeeze(1)
        # print("Computing accuracy")
        accuracies = self.accuracy_calculator.get_accuracy(
            test_embeddings, train_embeddings, test_labels, train_labels, False
        )
        # print(accuracies)
        self.log("val_acc", accuracies["precision_at_1"])
        # print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

    # def test_step(self, batch, batch_idx):
    #     train_embeddings, train_labels = self.get_all_embeddings(train_set, model)
    #     test_embeddings, test_labels = self.get_all_embeddings(test_set, model)
    #     train_labels = train_labels.squeeze(1)
    #     test_labels = test_labels.squeeze(1)
    #     print("Computing accuracy")
    #     accuracies = accuracy_calculator.get_accuracy(
    #         test_embeddings, train_embeddings, test_labels, train_labels, False
    #     )
    #     print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

    def configure_optimizers(self):
        # return optim.Adam(self.parameters(), lr=0.0001)
        
        return optim.AdamW(self.parameters(), lr=self.lr, amsgrad=False)
        # return {"lr_scheduler" : , "optimizer" : }




### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, 1) # [N, 30, 30, 32]
#         self.conv2 = nn.Conv2d(32, 64, 3, 1) # [N, 28, 28, 64]
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc1 = nn.Linear(9216, 128)

#     def forward(self, x):
#         x = self.conv1(x)# [N, 30, 30, 32]
#         x = F.relu(x)# [N, 30, 30, 32]
#         x = self.conv2(x)# [N, 28, 28, 64]
#         x = F.relu(x)# [N, 28, 28, 64]
#         x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         return x


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
# def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, labels) in enumerate(train_loader):
#         data, labels = data.to(device), labels.to(device)
#         optimizer.zero_grad()
#         embeddings = model(data)
#         Console().rule(title=f"EMBEDDINGS shape ===> {embeddings.shape}", style="bold magenta")
#         indices_tuple = mining_func(embeddings, labels)
#         Console().rule(title=f"INDICES shape ===> {indices_tuple}", style="bold magenta")
#         loss = loss_func(embeddings, labels, indices_tuple)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % 20 == 0:
#             print(
#                 "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
#                     epoch, batch_idx, loss, mining_func.num_triplets
#                 )
#             )


### convenient function from pytorch-metric-learning ###
# def get_all_embeddings(dataset, model):
#     tester = testers.BaseTester()
#     return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
# def test(train_set, test_set, model, accuracy_calculator):
#     model.eval()
#     train_embeddings, train_labels = get_all_embeddings(train_set, model)
#     test_embeddings, test_labels = get_all_embeddings(test_set, model)
#     train_labels = train_labels.squeeze(1)
#     test_labels = test_labels.squeeze(1)
#     print("Computing accuracy")
#     accuracies = accuracy_calculator.get_accuracy(
#         test_embeddings, train_embeddings, test_labels, train_labels, False
#     )
#     print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))


# device = torch.device("cuda")

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

batch_size = 200

dataset1 = datasets.MNIST(".", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST(".", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

# model = Net().to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# num_epochs = 3


### pytorch-metric-learning stuff ###
# distance = distances.CosineSimilarity()
# reducer = reducers.ThresholdReducer(low=0)
# loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
# mining_func = miners.TripletMarginMiner(
#     margin=0.2, distance=distance, type_of_triplets="semihard"
# )
# accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###


# for epoch in range(1, num_epochs + 1):
#     train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
#     test(dataset1, dataset2, model, accuracy_calculator)

# Config dictionary that will be logged to W&B.

# wandb.init()
wandb.login()



CONFIG = dict (
    dataset = "/content/MNIST",
    batch_size = batch_size,
    epochs = 10,
    precision = 16,
    amp_backend = "native",
    train_trf = transform,
    test_trf = transform,
    gpus = 1,
    checkpoint_callback = ModelCheckpoint(monitor='val_acc',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=False,
                                      filename='checkpoint/{epoch:03d}-{train_loss:.6f}-{val_acc:.6f}',
                                      verbose=True,
                                      mode='max'),

)

# checkpoint_callback = ModelCheckpoint(monitor='train_loss',
#                                       save_top_k=1,
#                                       save_last=True,
#                                       save_weights_only=False,
#                                       filename='checkpoint/{epoch:03d}-{train_loss:.4f}',
#                                       verbose=True,
#                                       mode='min')



wandb_logger = WandbLogger(project='train-siamese', 
                           config=CONFIG,
                           group='MNIST', 
                           job_type='using triplet-margin loss',
                           log_model="all")

model = recognition_model(dataset1, dataset2)
wandb_logger.watch(model, log="all")


trainer = Trainer(max_epochs = CONFIG["epochs"], 
                    precision=CONFIG["precision"], 
                    amp_backend=CONFIG["amp_backend"], 
                    gpus=CONFIG["gpus"],
            # checkpoint_callback=checkpoint_callback,
                    callbacks=[CONFIG["checkpoint_callback"]],
                    logger=wandb_logger,
                    weights_summary='top',)


trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)
# wandb_logger.unwatch(model)
wandb.finish() 


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Streaming output truncated to the last 5000 lines.

 76%|███████▋  | 1434/1875 [00:05<00:01, 280.50it/s]

 78%|███████▊  | 1469/1875 [00:05<00:01, 298.97it/s]

 80%|████████  | 1500/1875 [00:05<00:01, 301.79it/s]

 82%|████████▏ | 1531/1875 [00:05<00:01, 265.23it/s]

 84%|████████▎ | 1566/1875 [00:05<00:01, 286.43it/s]

 85%|████████▌ | 1598/1875 [00:05<00:00, 293.37it/s]

 87%|████████▋ | 1629/1875 [00:05<00:00, 274.81it/s]

 89%|████████▉ | 1666/1875 [00:05<00:00, 299.51it/s]

 91%|█████████ | 1697/1875 [00:06<00:00, 299.02it/s]

 92%|█████████▏| 1728/1875 [00:06<00:00, 265.27it/s]

 94%|█████████▍| 1761/1875 [00:06<00:00, 280.53it/s]

 96%|█████████▌| 1795/1875 [00:06<00:00, 295.29it/s]

 97%|█████████▋| 1826/1875 [00:06<00:00, 277.98it/s]

100%|██████████| 1875/1875 [00:06<00:00, 280.31it/s]


  0%|          | 0/313 [00:00<?, ?it/s]

  4%|▍         | 12/313 [00:00<00:02, 117.58it/s]

  9%|▉         | 29/313 [00:00<00:02, 128.40it/s]

 19%|█▉        | 60/313 [00:00<00:01, 203.66it/s

Validating: 0it [00:00, ?it/s]



  0%|          | 0/1875 [00:00<?, ?it/s]

  1%|          | 10/1875 [00:00<00:19, 97.72it/s]

  1%|▏         | 24/1875 [00:00<00:17, 105.21it/s]

  3%|▎         | 59/1875 [00:00<00:08, 204.11it/s]

  5%|▌         | 94/1875 [00:00<00:06, 257.62it/s]

  7%|▋         | 124/1875 [00:00<00:07, 236.57it/s]

  8%|▊         | 159/1875 [00:00<00:06, 269.43it/s]

 10%|█         | 192/1875 [00:00<00:05, 286.52it/s]

 12%|█▏        | 224/1875 [00:00<00:06, 264.03it/s]

 14%|█▎        | 255/1875 [00:01<00:05, 274.86it/s]

 15%|█▌        | 289/1875 [00:01<00:05, 291.96it/s]

 17%|█▋        | 322/1875 [00:01<00:05, 302.68it/s]

 19%|█▉        | 353/1875 [00:01<00:05, 271.58it/s]

 21%|██        | 387/1875 [00:01<00:05, 289.29it/s]

 22%|██▏       | 421/1875 [00:01<00:04, 303.32it/s]

 24%|██▍       | 453/1875 [00:01<00:05, 281.13it/s]

 26%|██▌       | 490/1875 [00:01<00:04, 303.71it/s]

 28%|██▊       | 524/1875 [00:01<00:04, 285.54it/s]

 30%|██▉       | 556/1875 [00:02<00:04, 293.34it/s]

 32%|██

In [3]:
! wandb.finish()

/bin/bash: -c: line 1: syntax error: unexpected end of file
